# Chapter 23: PL/pgSQL Essentials

PL/pgSQL is PostgreSQL's native procedural language, combining SQL's declarative power with imperative programming constructs. While SQL functions excel at data transformation, PL/pgSQL handles complex business logic, exception handling, and iterative processing. This chapter covers the essential patterns for writing maintainable, performant procedural code.

## 23.1 Language Fundamentals and Block Structure

PL/pgSQL is block-structured, with variable declarations preceding executable statements and optional exception handlers.

### 23.1.1 Block Architecture

```sql
-- Basic anonymous block (runs once, not stored):
DO $$
DECLARE
    -- Declaration section: variables, cursors, types
    user_count INTEGER;
    current_time TIMESTAMPTZ := NOW();
BEGIN
    -- Execution section: procedural logic
    SELECT COUNT(*) INTO user_count FROM users;
    RAISE NOTICE 'Found % users at %', user_count, current_time;
    
    -- Nested blocks for scoping:
    DECLARE
        local_var TEXT := 'inner scope';
    BEGIN
        RAISE NOTICE 'Inner: %', local_var;
        -- local_var not visible outside this block
    END;
    
EXCEPTION
    -- Exception handling (optional)
    WHEN division_by_zero THEN
        RAISE NOTICE 'Math error';
    WHEN OTHERS THEN
        RAISE NOTICE 'Error: %', SQLERRM;
END $$;

-- Stored function structure (reusable):
CREATE OR REPLACE FUNCTION get_user_stats(target_id BIGINT)
RETURNS TABLE(label TEXT, value TEXT) AS $$
DECLARE
    user_record RECORD;
    order_count BIGINT;
    total_spent DECIMAL;
BEGIN
    -- Retrieve user
    SELECT * INTO user_record FROM users WHERE id = target_id;
    
    IF NOT FOUND THEN
        RETURN;
    END IF;
    
    -- Calculate metrics
    SELECT COUNT(*), COALESCE(SUM(total), 0)
    INTO order_count, total_spent
    FROM orders
    WHERE user_id = target_id;
    
    -- Return results using RETURN QUERY
    RETURN QUERY SELECT 'Name'::TEXT, user_record.name::TEXT;
    RETURN QUERY SELECT 'Orders'::TEXT, order_count::TEXT;
    RETURN QUERY SELECT 'Spent'::TEXT, total_spent::TEXT;
END;
$$ LANGUAGE plpgsql STABLE;
```

### 23.1.2 Variable Declarations and Anchored Types

```sql
-- Anchored types (%TYPE): Automatically match column types
DECLARE
    v_user_id users.id%TYPE;           -- Matches BIGINT if id is BIGINT
    v_email users.email%TYPE;          -- Matches column type exactly
    v_created_at users.created_at%TYPE; -- TIMESTAMPTZ with precision
    
    -- Constants
    c_tax_rate CONSTANT DECIMAL := 0.08;
    c_max_retries CONSTANT INTEGER := 3;
    
    -- Row type (matches entire table structure)
    v_user_record users%ROWTYPE;
    
    -- Record type (flexible structure)
    v_custom_record RECORD;
    
    -- Arrays
    v_id_list BIGINT[];
    v_prices DECIMAL(10,2)[];
    
    -- Default values
    v_status TEXT := 'pending';
    v_count INTEGER DEFAULT 0;
    
    -- NOT NULL constraints
    v_required_id BIGINT NOT NULL := 0;
BEGIN
    -- Using anchored types prevents schema drift
    SELECT id, email, created_at 
    INTO v_user_id, v_email, v_created_at
    FROM users 
    WHERE id = 123;
    
    -- Row type assignment
    SELECT * INTO v_user_record FROM users WHERE id = 123;
    RAISE NOTICE 'User: %, Email: %', v_user_record.name, v_user_record.email;
END $$;

-- Benefits of %TYPE and %ROWTYPE:
-- 1. If column type changes (e.g., INTEGER to BIGINT), function recompiles correctly
-- 2. No manual type synchronization needed during schema migrations
-- 3. Self-documenting: code clearly shows data origins
```

## 23.2 Control Structures

PL/pgSQL provides standard imperative control flow with SQL-integrated conditionals.

### 23.2.1 Conditional Logic

```sql
-- IF/THEN/ELSEIF/ELSE structure:
CREATE OR REPLACE FUNCTION calculate_shipping(
    weight DECIMAL,
    distance INTEGER,
    method TEXT
) RETURNS DECIMAL AS $$
DECLARE
    base_rate DECIMAL;
    weight_factor DECIMAL;
BEGIN
    IF weight <= 0 THEN
        RAISE EXCEPTION 'Weight must be positive: %', weight;
    END IF;
    
    -- Method selection
    IF method = 'express' THEN
        base_rate := 25.00;
        weight_factor := 0.75;
    ELSIF method = 'standard' THEN
        base_rate := 10.00;
        weight_factor := 0.40;
    ELSIF method = 'economy' THEN
        base_rate := 5.00;
        weight_factor := 0.25;
    ELSE
        RAISE EXCEPTION 'Unknown shipping method: %', method;
    END IF;
    
    RETURN base_rate + (weight * weight_factor * (distance / 100.0));
END;
$$ LANGUAGE plpgsql IMMUTABLE;

-- Simple CASE expression (SQL standard, works in PL/pgSQL):
DECLARE
    status_code INTEGER := 2;
    status_text TEXT;
BEGIN
    status_text := CASE status_code
        WHEN 1 THEN 'Pending'
        WHEN 2 THEN 'Processing'
        WHEN 3 THEN 'Shipped'
        WHEN 4 THEN 'Delivered'
        ELSE 'Unknown'
    END;
    
    -- Searched CASE (conditions instead of values):
    status_text := CASE
        WHEN status_code = 1 THEN 'Pending'
        WHEN status_code BETWEEN 2 AND 3 THEN 'In Progress'
        WHEN status_code >= 4 THEN 'Complete'
        ELSE 'Invalid'
    END;
END $$;

-- COALESCE and NULLIF (prefer over simple IF for null handling):
DECLARE
    display_name TEXT;
BEGIN
    -- IF/THEN equivalent: unnecessarily verbose
    -- IF user_nickname IS NOT NULL THEN display_name := user_nickname;
    -- ELSE display_name := user_full_name; END IF;
    
    -- Better:
    display_name := COALESCE(user_nickname, user_full_name, 'Anonymous');
END $$;
```

### 23.2.2 Looping Constructs

```sql
-- Simple LOOP (infinite until EXIT):
DECLARE
    counter INTEGER := 0;
BEGIN
    LOOP
        counter := counter + 1;
        EXIT WHEN counter > 100;  -- Exit condition
        CONTINUE WHEN counter % 10 = 0;  -- Skip multiples of 10
        -- Process...
    END LOOP;
END $$;

-- WHILE loop:
DECLARE
    batch_size INTEGER := 1000;
    processed INTEGER := 0;
BEGIN
    WHILE processed < 10000 LOOP
        -- Process batch
        UPDATE tasks SET status = 'done' 
        WHERE id IN (
            SELECT id FROM tasks 
            WHERE status = 'pending' 
            LIMIT batch_size
        );
        
        GET DIAGNOSTICS processed = ROW_COUNT;
        COMMIT;  -- If in procedure context
    END LOOP;
END $$;

-- FOR loop with integer range:
DECLARE
    i INTEGER;
BEGIN
    FOR i IN 1..10 LOOP
        RAISE NOTICE 'Iteration %', i;
    END LOOP;
    
    -- Reverse:
    FOR i IN REVERSE 10..1 LOOP
        -- Counts down
    END LOOP;
    
    -- With step (PostgreSQL 11+):
    FOR i IN 1..100 BY 5 LOOP
        -- 1, 6, 11, 16...
    END LOOP;
END $$;

-- FOREACH for arrays:
DECLARE
    sku_list TEXT[] := ARRAY['SKU-001', 'SKU-002', 'SKU-003'];
    current_sku TEXT;
BEGIN
    FOREACH current_sku IN ARRAY sku_list LOOP
        UPDATE inventory SET stock = stock - 1 WHERE sku = current_sku;
    END LOOP;
END $$;
```

## 23.3 Cursor Processing

Cursors provide controlled iteration over result sets. In PL/pgSQL, implicit cursors (FOR loops) are preferred over explicit cursors for readability and safety.

### 23.3.1 Implicit Cursors (FOR Loop)

```sql
-- The PL/pgSQL FOR loop automatically creates, opens, fetches, and closes cursor:
CREATE OR REPLACE FUNCTION process_pending_orders()
RETURNS INTEGER AS $$
DECLARE
    processed_count INTEGER := 0;
    order_rec RECORD;  -- Can use specific table%ROWTYPE too
BEGIN
    -- Implicit cursor: orders_cursor is automatically managed
    FOR order_rec IN 
        SELECT order_id, customer_id, total 
        FROM orders 
        WHERE status = 'pending' 
        ORDER BY created_at
        LIMIT 1000
    LOOP
        -- Process each order
        UPDATE inventory 
        SET reserved = reserved + 1 
        WHERE product_id IN (
            SELECT product_id FROM order_items WHERE order_id = order_rec.order_id
        );
        
        UPDATE orders SET status = 'processing' WHERE order_id = order_rec.order_id;
        processed_count := processed_count + 1;
        
        -- Exit early if needed:
        IF processed_count >= 100 THEN
            EXIT;
        END IF;
    END LOOP;
    
    RETURN processed_count;
END;
$$ LANGUAGE plpgsql;

-- Reverse iteration (if index supports backwards scan):
FOR order_rec IN 
    SELECT * FROM orders 
    WHERE created_at > '2024-01-01'
    ORDER BY created_at DESC
LOOP
    -- Processes from newest to oldest
END LOOP;
```

### 23.3.2 Explicit Cursors (When Needed)

```sql
-- Explicit cursors required when:
-- 1. Passing cursor between functions
-- 2. Partial fetching (not processing all rows)
-- 3. Dynamic SQL with cursors

CREATE OR REPLACE FUNCTION fetch_page(
    cursor_name TEXT,
    page_size INTEGER
) RETURNS SETOF orders AS $$
DECLARE
    cur REFCURSOR;
    rec orders%ROWTYPE;
    i INTEGER := 0;
BEGIN
    -- Open cursor (must be declared in calling context or here)
    OPEN cur FOR SELECT * FROM orders WHERE status = 'pending';
    
    -- Fetch specific number of rows
    WHILE i < page_size LOOP
        FETCH cur INTO rec;
        EXIT WHEN NOT FOUND;
        RETURN NEXT rec;
        i := i + 1;
    END LOOP;
    
    -- Cursor remains open for next call (if using session cursor)
    -- CLOSE cur; -- Only if done
    RETURN;
END;
$$ LANGUAGE plpgsql;

-- Refcursor for returning cursor to caller:
CREATE OR REPLACE FUNCTION get_order_cursor(min_amount DECIMAL)
RETURNS REFCURSOR AS $$
DECLARE
    cur REFCURSOR := 'order_cur';  -- Named cursor
BEGIN
    OPEN cur FOR 
        SELECT * FROM orders 
        WHERE total > min_amount 
        ORDER BY created_at;
    RETURN cur;
END;
$$ LANGUAGE plpgsql;

-- Usage from application:
-- BEGIN;
-- SELECT get_order_cursor(100.00);
-- FETCH 10 FROM order_cur;
-- CLOSE order_cur;
-- COMMIT;
```

### 23.3.3 Cursor FOR UPDATE

```sql
-- Locking rows as you fetch them (pessimistic concurrency):
CREATE OR REPLACE FUNCTION process_with_locks()
RETURNS VOID AS $$
DECLARE
    rec RECORD;
BEGIN
    FOR rec IN 
        SELECT * FROM tasks 
        WHERE status = 'queued' 
        ORDER BY priority DESC, created_at
        FOR UPDATE SKIP LOCKED  -- Skip rows locked by other sessions
        LIMIT 10
    LOOP
        -- Row is locked until transaction ends
        PERFORM pg_sleep(1);  -- Simulate work
        UPDATE tasks SET status = 'running' WHERE CURRENT OF <cursor>;
        -- Note: CURRENT OF only works with explicit cursors, use PK in practice:
        -- UPDATE tasks SET status = 'running' WHERE id = rec.id;
    END LOOP;
END;
$$ LANGUAGE plpgsql;

-- Important: FOR UPDATE cursors hold locks for transaction duration
-- Keep transactions short when using this pattern
```

## 23.4 Dynamic SQL and Execution

Dynamic SQL executes statements constructed at runtime, essential for DDL and variable table/column names.

### 23.4.1 EXECUTE with Parameter Binding

```sql
-- Basic EXECUTE (vulnerable to injection if not careful):
CREATE OR REPLACE FUNCTION unsafe_query(table_name TEXT, column_value TEXT)
RETURNS INTEGER AS $$
DECLARE
    result INTEGER;
BEGIN
    -- NEVER DO THIS: SQL injection vulnerability
    EXECUTE 'SELECT COUNT(*) FROM ' || table_name || ' WHERE name = ''' || column_value || ''''
    INTO result;
    RETURN result;
END;
$$ LANGUAGE plpgsql;

-- Safe EXECUTE using format() and USING:
CREATE OR REPLACE FUNCTION count_by_status(
    target_table TEXT,
    status_value TEXT
) RETURNS INTEGER AS $$
DECLARE
    query TEXT;
    result INTEGER;
BEGIN
    -- format() with %I (identifier) and %L (literal)
    query := format(
        'SELECT COUNT(*) FROM %I WHERE status = $1',
        target_table  -- %I quotes identifiers, prevents injection
    );
    
    -- USING binds parameters (safe from injection)
    EXECUTE query INTO result USING status_value;
    
    RETURN result;
END;
$$ LANGUAGE plpgsql STABLE;

-- Dynamic DDL example:
CREATE OR REPLACE FUNCTION create_audit_table(source_table TEXT)
RETURNS VOID AS $$
BEGIN
    EXECUTE format(
        'CREATE TABLE IF NOT EXISTS %I_audit (
            operation TEXT,
            stamp TIMESTAMPTZ,
            userid TEXT,
            old_data JSONB,
            new_data JSONB
        )',
        source_table
    );
    
    -- Create index
    EXECUTE format(
        'CREATE INDEX idx_%s_audit_stamp ON %I_audit(stamp)',
        source_table, source_table
    );
END;
$$ LANGUAGE plpgsql;

-- format() specifiers:
-- %s - String (literal, quoted)
-- %I - Identifier (table/column names, quoted appropriately)
-- %L - SQL literal (quoted string)
-- %% - Literal %
```

### 23.4.2 RETURN QUERY with Dynamic SQL

```sql
-- Returning results from dynamic queries:
CREATE OR REPLACE FUNCTION search_dynamic(
    search_term TEXT,
    target_column TEXT
) RETURNS SETOF users AS $$
BEGIN
    RETURN QUERY EXECUTE format(
        'SELECT * FROM users WHERE %I ILIKE $1',
        target_column
    ) USING '%' || search_term || '%';
END;
$$ LANGUAGE plpgsql STABLE;

-- Note: Return type must match actual query structure
-- For variable result shapes, use RETURNS SETOF RECORD with caller specifying structure
```

## 23.5 Error Handling Deep Dive

PL/pgSQL provides sophisticated exception handling with specific error codes and stack trace information.

### 23.5.1 Exception Block Structure

```sql
-- Complete exception handling:
CREATE OR REPLACE FUNCTION safe_divide(numerator DECIMAL, denominator DECIMAL)
RETURNS DECIMAL AS $$
DECLARE
    result DECIMAL;
BEGIN
    result := numerator / denominator;
    RETURN result;
    
EXCEPTION
    -- Specific named conditions
    WHEN division_by_zero THEN
        RAISE NOTICE 'Division by zero detected';
        RETURN NULL;
        
    WHEN numeric_value_out_of_range THEN
        RAISE WARNING 'Numeric overflow in calculation';
        RETURN NULL;
        
    -- Specific SQLSTATE codes (5 character)
    WHEN SQLSTATE '23505' THEN  -- unique_violation
        RAISE EXCEPTION 'Duplicate value detected';
        
    -- User-defined exceptions (raise with specific code)
    WHEN SQLSTATE 'P0001' THEN  -- default for RAISE EXCEPTION
        RAISE NOTICE 'Custom error occurred: %', SQLERRM;
        
    -- Catch-all
    WHEN OTHERS THEN
        -- Log error details
        RAISE WARNING 'Error %: %', SQLSTATE, SQLERRM;
        RAISE WARNING 'Detail: %', PG_EXCEPTION_DETAIL;
        RAISE WARNING 'Hint: %', PG_EXCEPTION_HINT;
        RAISE WARNING 'Context: %', PG_EXCEPTION_CONTEXT;
        
        -- Re-raise to caller if unrecoverable
        RAISE;
END;
$$ LANGUAGE plpgsql;

-- Common SQLSTATE codes to handle:
-- 23502 - not_null_violation
-- 23503 - foreign_key_violation  
-- 23505 - unique_violation
-- 23514 - check_violation
-- 42703 - undefined_column
-- 42P01 - undefined_table
-- 57014 - query_canceled
```

### 23.5.2 Custom Exceptions and RAISE

```sql
-- Raising exceptions with context:
CREATE OR REPLACE FUNCTION validate_order(order_id BIGINT)
RETURNS VOID AS $$
DECLARE
    order_total DECIMAL;
    customer_status TEXT;
BEGIN
    SELECT total, c.status 
    INTO order_total, customer_status
    FROM orders o
    JOIN customers c ON c.id = o.customer_id
    WHERE o.id = order_id;
    
    IF NOT FOUND THEN
        RAISE EXCEPTION 'Order % not found', order_id
            USING ERRCODE = 'no_data_found',
                  HINT = 'Verify the order ID and try again';
    END IF;
    
    IF customer_status = 'suspended' THEN
        RAISE EXCEPTION 'Cannot process order for suspended customer'
            USING ERRCODE = 'check_violation',  -- 23514
                  DETAIL = format('Customer status is %s', customer_status),
                  HINT = 'Contact customer support to reactivate account';
    END IF;
    
    IF order_total > 10000 THEN
        RAISE EXCEPTION 'Order exceeds approval limit'
            USING ERRCODE = 'limit_exceeded',  -- Custom code (P0002-P9999)
                  DETAIL = format('Order total %s exceeds limit 10000', order_total);
    END IF;
END;
$$ LANGUAGE plpgsql;

-- RAISE levels (in order of severity):
-- DEBUG1..DEBUG5 - Detailed debugging info
-- LOG - Log to server log only
-- INFO - Informational message
-- NOTICE - Default, sent to client
-- WARNING - Warning message
-- EXCEPTION - Throws error (transaction rolls back to savepoint)
```

## 23.6 Performance Considerations

PL/pgSQL introduces context switches between the PL engine and SQL executor. Minimizing these switches is key to performance.

### 23.6.1 Context Switch Overhead

```sql
-- Anti-pattern: Row-by-row processing (slowest):
CREATE OR REPLACE FUNCTION update_prices_slow()
RETURNS VOID AS $$
DECLARE
    rec RECORD;
BEGIN
    FOR rec IN SELECT id, price FROM products LOOP
        -- Context switch per row: PL -> SQL -> PL
        UPDATE products 
        SET price = price * 1.1 
        WHERE id = rec.id;
    END LOOP;
END;
$$ LANGUAGE plpgsql;

-- Better: Single SQL statement (no PL/pgSQL needed):
UPDATE products SET price = price * 1.1 WHERE category = 'electronics';

-- If logic required per row, batch in groups:
CREATE OR REPLACE FUNCTION update_prices_batched()
RETURNS VOID AS $$
DECLARE
    batch_size CONSTANT INTEGER := 1000;
    rows_updated INTEGER;
BEGIN
    LOOP
        -- Single UPDATE per batch (amortizes context switch cost)
        UPDATE products 
        SET price = price * 1.1 
        WHERE id IN (
            SELECT id 
            FROM products 
            WHERE needs_update = true 
            LIMIT batch_size
        );
        
        GET DIAGNOSTICS rows_updated = ROW_COUNT;
        EXIT WHEN rows_updated = 0;
        
        COMMIT;  -- If procedure, to release locks
    END LOOP;
END;
$$ LANGUAGE plpgsql;
```

### 23.6.2 Bulk Operations with Arrays

```sql
-- Passing arrays to process in batches:
CREATE OR REPLACE FUNCTION process_order_ids(order_ids BIGINT[])
RETURNS TABLE(order_id BIGINT, success BOOLEAN, message TEXT) AS $$
DECLARE
    current_id BIGINT;
    start_time TIMESTAMPTZ;
BEGIN
    FOREACH current_id IN ARRAY order_ids LOOP
        BEGIN
            start_time := clock_timestamp();
            
            -- Process single order
            PERFORM process_single_order(current_id);
            
            order_id := current_id;
            success := true;
            message := 'Processed in ' || extract(epoch from clock_timestamp() - start_time) || 's';
            
        EXCEPTION WHEN OTHERS THEN
            order_id := current_id;
            success := false;
            message := SQLERRM;
        END;
        
        RETURN NEXT;
    END LOOP;
END;
$$ LANGUAGE plpgsql;

-- Usage:
SELECT * FROM process_order_ids(ARRAY[1, 2, 3, 4, 5]);
```

### 23.6.3 Diagnosing Performance

```sql
-- Timing sections of code:
DECLARE
    v_start TIMESTAMPTZ;
    v_elapsed INTERVAL;
BEGIN
    v_start := clock_timestamp();
    
    -- Code to time
    PERFORM heavy_operation();
    
    v_elapsed := clock_timestamp() - v_start;
    RAISE NOTICE 'Operation took %', v_elapsed;
    
    -- Using EXPLAIN inside PL/pgSQL (PostgreSQL 14+):
    -- FOR rec IN EXPLAIN (ANALYZE, FORMAT JSON) SELECT * FROM large_table LOOP
    --     RAISE NOTICE 'Plan: %', rec.QUERY PLAN;
    -- END LOOP;
END $$;
```

## 23.7 Best Practices and Patterns

### 23.7.1 Function Design Guidelines

```sql
-- 1. Use STRICT for simple null handling:
CREATE FUNCTION safe_add(a INTEGER, b INTEGER)
RETURNS INTEGER AS $$
    SELECT a + b;
$$ LANGUAGE SQL IMMUTABLE STRICT;
-- STRICT means return NULL immediately if any arg is NULL
-- (Equivalent to checking IF a IS NULL OR b IS NULL THEN RETURN NULL)

-- 2. Check FOUND after SELECT INTO:
SELECT * INTO rec FROM users WHERE id = v_id;
IF NOT FOUND THEN
    RAISE EXCEPTION 'User % not found', v_id;
END IF;

-- 3. Use RETURN QUERY for set returning:
CREATE FUNCTION get_active_users()
RETURNS SETOF users AS $$
BEGIN
    -- Good: Single SQL statement, optimizer can inline
    RETURN QUERY SELECT * FROM users WHERE active = true;
    
    -- Bad: Row-by-row return next (unless complex logic required)
    -- FOR rec IN SELECT * FROM users WHERE active = true LOOP
    --     RETURN NEXT rec;
    -- END LOOP;
END;
$$ LANGUAGE plpgsql STABLE;

-- 4. Avoid side effects in volatile functions called in queries:
-- SELECT expensive_audit_function(column) FROM large_table;
-- If function inserts to audit table, called once per row = slow
-- Better: Use triggers or batch process

-- 5. Document assumptions with ASSERT (debugging):
CREATE FUNCTION divide(a DECIMAL, b DECIMAL)
RETURNS DECIMAL AS $$
BEGIN
    ASSERT b != 0, 'Division by zero assertion';
    RETURN a / b;
END;
$$ LANGUAGE plpgsql;
-- Assertions can be disabled globally for production (check_function_bodies)
```

### 23.7.2 Security Patterns

```sql
-- Defensive programming against SQL injection:
CREATE FUNCTION search_users(search_criteria JSONB)
RETURNS SETOF users AS $$
DECLARE
    where_clause TEXT := '';
    valid_columns TEXT[] := ARRAY['name', 'email', 'status'];
    key TEXT;
    value TEXT;
BEGIN
    -- Whitelist column names
    FOR key, value IN SELECT * FROM jsonb_each_text(search_criteria) LOOP
        IF NOT key = ANY(valid_columns) THEN
            RAISE EXCEPTION 'Invalid search column: %', key;
        END IF;
        
        where_clause := where_clause || format(' AND %I = %L', key, value);
    END LOOP;
    
    -- Remove leading AND
    where_clause := ltrim(where_clause, ' AND');
    
    RETURN QUERY EXECUTE 
        'SELECT * FROM users WHERE ' || where_clause;
END;
$$ LANGUAGE plpgsql STABLE;

-- Search path safety (always set explicitly):
CREATE FUNCTION get_secure_data()
RETURNS TABLE(id INTEGER, data TEXT) AS $$
BEGIN
    RETURN QUERY SELECT id, data FROM secure_schema.sensitive_table;
END;
$$ LANGUAGE plpgsql STABLE SECURITY DEFINER
SET search_path = secure_schema, pg_temp;
```

---

## Chapter Summary

In this chapter, you learned:

1. **Block Structure**: PL/pgSQL uses declaration/begin/exception blocks with scoped variables. Always use anchored types (`%TYPE`, `%ROWTYPE`) to ensure schema changes don't break functions.

2. **Control Flow**: Use simple SQL `CASE` and `COALESCE` where possible instead of procedural `IF/THEN`. For loops should iterate over queries (implicit cursors) rather than building explicit cursors unless passing between functions or implementing pagination.

3. **Cursors**: Implicit `FOR rec IN query LOOP` handles opening, fetching, and closing automatically. Use `FOR UPDATE SKIP LOCKED` for queue processing. Explicit cursors (`REFCURSOR`) are only needed for inter-function cursor passing or dynamic result sets.

4. **Dynamic SQL**: Use `EXECUTE ... USING` with `format()` specifiers (`%I` for identifiers, `%L` for literals) to prevent SQL injection. Never concatenate user input directly into query strings.

5. **Error Handling**: Trap specific `SQLSTATE` codes or named conditions (`division_by_zero`, `unique_violation`) rather than `WHEN OTHERS` whenever possible. Use `RAISE` with `ERRCODE`, `DETAIL`, and `HINT` for actionable error messages.

6. **Performance**: Minimize context switches between PL and SQL engines. Avoid row-by-row updates in loops—use single SQL statements or batched updates with `ctid` or primary key ranges. Each `EXECUTE` or DML statement in a loop is a context switch.

**Next:** In Chapter 24, we will explore Triggers—covering types and timing, auditing implementations, denormalized cache updates, and strategies for avoiding hidden complexity while maintaining data integrity.